In [4]:
import os
import matplotlib
import vame
import numpy as np
import glob
from vame.util import auxiliary as aux
from vame.analysis import behavior_structure as bs
from vame.custom import helperFunctions as hf
from vame.custom import ACWS_videowriter as avw
import polars as pl
import pandas as pd
from vame.util import auxiliary as aux
from vame.analysis import behavior_structure as bs
from vame.custom import helperFunctions as hf
from vame.custom import ALR_helperFunctions as AlHf
from vame.custom import ALR_plottingFunctions as AlPf
from vame.custom import ACWS_videowriter as avw
from vame.custom import ALR_kinematics as kin
from vame.custom import ALR_analysis as an
from vame.analysis import community_analysis as ca
from importlib import reload
from icecream import ic
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import six


# Analysis
## Checking the Normality of the Data

In [5]:
reload(AlHf)
#df = pd.read_csv("/Users/adanredwine/Downloads/CombinedMotifUsage-drawn-sweep-88-40-200_100prcnt.csv")
df = pd.read_csv("/Users/adanredwine/Desktop/ALR_1-VAME-Project-Feb3-2023/CombinedMotifUsage-kmeans-30.csv")
meta_data = AlHf.create_meta_data_df(df)
normality_results_Group = AlHf.check_normality_by_group(df, meta_data)
normality_results_GroupTime = AlHf.check_normality_by_group_and_time(df, meta_data)
normality_results_GroupTimeMotif = AlHf.check_normality_group_timepoint_motif(df, meta_data)
normality_results_StateTimeMotif = AlHf.check_normality_state_timepoint_motif(df, meta_data)

In [6]:
normality_results_Group

,Group,W,p
0,Sham,0.740767,0.0
1,ABX,0.735181,0.0
2,Injured,0.825460,0.0
3,Treated,0.787291,0.0


In [7]:
normality_results_GroupTime

,Group,Time_Point,W,p,Note
0,Sham,Baseline_1,0.864440,3.493903e-17,NaN
1,Sham,Baseline_2,0.786921,2.297405e-21,NaN
2,Sham,Week_02,0.709611,1.639493e-24,NaN
3,Sham,Week_04,0.694650,4.811304e-25,NaN
4,Sham,Week_06,0.700416,7.674122e-25,NaN
5,Sham,Week_08,0.761729,1.786731e-22,NaN
6,Sham,Week_11,0.755931,1.022052e-22,NaN
7,Sham,Week_13,0.755759,1.005356e-22,NaN
8,Sham,Week_15,0.717889,3.299606e-24,NaN
9,Sham,Drug_Trt,0.696749,5.698159e-25,NaN


In [8]:
total_motifs_Group = len(normality_results_GroupTimeMotif)
normal_motifs_Group = normality_results_GroupTimeMotif[normality_results_GroupTimeMotif['p'] > 0.05]
non_normal_motifs_Group = normality_results_GroupTimeMotif[normality_results_GroupTimeMotif['p'] <= 0.05]
percent_normal_Group = (len(normal_motifs_Group) / total_motifs_Group) * 100

total_motifs_State = len(normality_results_StateTimeMotif)
normal_motifs_State = normality_results_StateTimeMotif[normality_results_StateTimeMotif['p'] > 0.05]
non_normal_motifs_State = normality_results_StateTimeMotif[normality_results_StateTimeMotif['p'] <= 0.05]
percent_normal_State = (len(normal_motifs_State) / total_motifs_State) * 100


In [9]:
percent_normal_Group 

55.75

In [10]:
percent_normal_State

44.583333333333336

In both observation angles, only about half of the data is normal. A log transformation will be done to the values. Additionally, log values will be normalized to baseline and sham values.

In [ ]:
AlPf.plot_distribution_of_motif_3(df, meta_data, motif=None)

In [11]:
#clusters = np.load("/Users/adanredwine/Desktop/ALR_1-VAME-Project-Feb3-2023/clusters_all_data-11.npy")
clusters = np.load("/Users/adanredwine/Desktop/ALR_1-VAME-Project-Feb3-2023/clusters_wSpeed_all_data-11_kmeans-30.npy")
clustered_motifs =  AlHf.group_motifs_by_cluster(clusters)
df_long = AlHf.melt_df(df, meta_data)
df_long = AlHf.assign_clusters(df_long, clustered_motifs)

In [13]:
reload(AlHf)
df_long_norm_base = AlHf.normalize_to_baseline(df_long)
df_long_logNorm_base = AlHf.normalize_to_baseline_log(df_long)
df_long_norm_sham = AlHf.normalize_to_baseline_sham(df_long)
df_long_logNorm_sham = AlHf.normalize_to_baseline_sham_log(df_long)

Log transformation is a powerful statistical tool used to stabilize the variance, make the data more closely conform to normality, and reduce the effects of outliers. Here's how it affects the data:

1. Normalization: Log transformation can help in normalizing data that is positively skewed. Many biological, economic, and other data sets naturally exhibit a right-skewed distribution, and log transformation can make such data conform more closely to a normal distribution.

2. Variance Stabilization: In some datasets, the variance increases with the mean. Log transformation can stabilize the variance across levels of an independent variable, which is an assumption of many parametric statistical tests.

3. Multiplicative to Additive: Log transformation turns multiplicative relationships into additive relationships, which can simplify the modeling of relationships between variables.

4. Handling Outliers: It can diminish the influence of outliers due to the compression effect on the scale. Outliers will have less impact on the parameter estimates in a statistical model.

5. Interpretation Changes: After log transformation, the interpretation of the results changes. For example, in a log-transformed linear regression, the coefficients represent the percentage change in the dependent variable for a one percent change in the independent variable, rather than the absolute change.

6. Zero Values: Log transformation cannot be directly applied to zero or negative values since the logarithm of zero and negative numbers is undefined. In such cases, a constant may be added to all values before transformation.

7. Back-Transformation: After analysis, results may need to be back-transformed to the original scale, which can be non-trivial, especially for measures of variability or for the interpretation of interaction effects.

It's important to note that log transformation is not always the appropriate or necessary transformation, and its suitability depends on the specific characteristics and requirements of the data and analysis. Always visualize and test your data before and after transformation to ensure that it meets the necessary assumptions for your statistical tests.

Whether to use normalized values or raw values for statistical tests depends on the context of your study and what you aim to compare.

Here are some considerations to help you decide:

1. Raw Values: Using raw values can be appropriate when you want to assess the actual measured quantities and their differences are meaningful. Raw values are also useful when you want to maintain the original scale and variance of the data, which can be important for certain statistical tests.

2. Normalized Values: Normalization is often used when you want to control for baseline differences between subjects or groups, or when you want to compare relative changes rather than absolute values. For example, if you're interested in the percentage change from baseline or if the baseline values vary significantly between subjects, normalization can help to account for these differences.

For repeated measures ANOVA and mixed-effects models, which are designed to handle repeated measurements from the same subjects, using normalized values can be beneficial if you're interested in relative changes over time and if there's a need to control for individual baseline differences.

For non-parametric tests like the Friedman test or Kruskal-Wallis test, which are less sensitive to the scale of the data, normalized values can be used if the goal is to compare relative changes or rankings.

In summary, if your study's focus is on the change relative to a baseline or control condition, and you want to account for individual variability at the starting point, then normalized values may be more appropriate. However, if the actual measured values and their scale are important for the research question, then raw values should be used.

Always consider the assumptions and requirements of the statistical test you're using, and ensure that the method of normalization is appropriate for the test. If in doubt, it's a good idea to consult with a statistician or methodologist.

In [14]:
reload(AlHf)
result = AlHf.run_mixed_effects_model(df_long, dependent_var='Normalized_Value', groups='Animal_ID', re_formula='1', fe_formula='Time_Point * Group', maxiter=100)
print(result.summary())

IndexError: index 13968 is out of bounds for axis 0 with size 13968

In [15]:
reload(AlHf)
result_logNorm_base = AlHf.run_mixed_effects_model(df_long_logNorm_base, dependent_var='Log_Normalized_Value', groups='Animal_ID', re_formula='1', fe_formula='Time_Point * Group', maxiter=100)
print(result_logNorm_base.summary())

/Users/adanredwine/miniconda3/envs/VAME-2/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/adanredwine/miniconda3/envs/VAME-2/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Converged with lbfgs
                       Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       Log_Normalized_Value
No. Observations:       14040         Method:                   REML                
No. Groups:             48            Scale:                    1.1807              
Min. group size:        270           Log-Likelihood:           -21127.5331         
Max. group size:        300           Converged:                Yes                 
Mean group size:        292.5                                                       
------------------------------------------------------------------------------------
                                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------
Intercept                                  0.029    0.057  0.505 0.614 -0.083  0.141
Time_Point[T.Baseline_2]                  -0.000    0.081 -0.000 1.0

/Users/adanredwine/miniconda3/envs/VAME-2/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2706: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))


In [16]:
reload(AlHf)
result_logNorm_sham = AlHf.run_mixed_effects_model(df_long_logNorm_sham, dependent_var='Log_Normalized_Value', groups='Animal_ID', re_formula='1', fe_formula='Time_Point * Group', maxiter=100)
print(result_logNorm_sham.summary())

/Users/adanredwine/miniconda3/envs/VAME-2/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


Converged with lbfgs
                       Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       Log_Normalized_Value
No. Observations:       14040         Method:                   REML                
No. Groups:             48            Scale:                    0.9990              
Min. group size:        270           Log-Likelihood:           -20002.2062         
Max. group size:        300           Converged:                Yes                 
Mean group size:        292.5                                                       
------------------------------------------------------------------------------------
                                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------
Intercept                                 -0.007    0.067 -0.108 0.914 -0.139  0.125
Time_Point[T.Baseline_2]                  -0.121    0.074 -1.627 0.1

/Users/adanredwine/miniconda3/envs/VAME-2/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2706: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))


### **Notes on the mixed effects models:**


<span style="color:blue;">Formula: </span> Adjust the fe_formula and re_formula according to specific model needs. The fe_formula represents the fixed effects you want to include in the model, and re_formula represents the random effects.

*In mixed-effects models, the terms "fixed effects" and "random effects" have specific meanings:*
**Fixed Effects:**
Fixed effects are the variables of interest that you expect to have a systematic and consistent impact on the dependent variable across all units of observation. They are the factors you believe will influence your outcome variable in a predictable way. In your model, fixed effects would include:

- Time Points: These represent the different time measurements in your study. You are likely interested in how the dependent variable changes over these time points.
- Groups: These represent the different treatment or condition groups in your study. You are likely interested in the differences in the dependent variable between these groups.

The coefficients for fixed effects are estimated directly and apply to the entire population. They are akin to the coefficients in a standard regression model.
**Random Effects:**
Random effects are variables that introduce variability into the model but are not the primary variables of interest. Instead, they account for the random variation at different levels of the data hierarchy. In your model, the random effects might include:

- Animal_ID: This represents the individual animals in your study. Each animal might respond differently to the treatments or conditions, and these individual differences are not the main focus of your study. However, they are important to account for because they can influence the dependent variable.

Random effects are assumed to be randomly distributed across different levels of the data (e.g., subjects or clusters). They allow the model to account for the fact that observations within the same group (like the same animal) are not independent and may be more similar to each other than to observations from different groups.

In summary, fixed effects are your main variables of interest that you hypothesize will affect the outcome consistently, while random effects are used to model the variability in the data that is not explained by the fixed effects. Random effects allow for individual differences and are useful when data is collected in a nested or hierarchical structure (like multiple measurements from the same subjects).


```diff
+ ### How to Color Text in Markdown
+ 
+ To color text in markdown, you typically need to use HTML tags, as standard Markdown does not support text color directly. Here's an example of how to change the color of text using HTML:
+
+ ```html
+ <span style="color:red;">This text will be red</span>
+ ```
+
+ Replace `red` with any other color name or hex code to change the text color accordingly.
```


The output you're seeing is the summary of a mixed-effects linear model that you've run using the statsmodels library in Python. This model is used to analyze data that has both fixed effects (such as time points and treatment groups) and random effects (such as individual variability in animals). Here's a breakdown of the output and the warnings:

<span style="color:red;">Warnings:</span>

1. Convergence Warning: This warning indicates that the optimization algorithm used to find the maximum likelihood estimates did not converge. This means that the algorithm did not successfully find the best-fit parameters within the specified number of iterations or tolerance levels. It suggests that the results may not be reliable.

2. Retrying MixedLM optimization with lbfgs: This is a follow-up message indicating that the algorithm is retrying with a different optimization method (lbfgs) to attempt to find a solution.

3. Random effects covariance is singular: This warning suggests that the variance-covariance matrix for the random effects has some issues, possibly due to a lack of variability in the random effects or overparameterization of the model.
**Model Summary:**
- Dependent Variable: The outcome variable that the model is trying to predict (Normalized_Value).
- Method: The method used to estimate the parameters (REML stands for Restricted Maximum Likelihood).
- No. Observations: The number of data points used in the model (18,720).
- No. Groups: The number of groups for the random effects (48, which likely corresponds to the number of Animal_IDs).
- Scale: The estimated variance of the residuals from the model.
- Log-Likelihood: The log-likelihood of the model, which is a measure of model fit (higher is generally better).
- Converged: Indicates whether the optimization algorithm successfully converged (Yes in this case, despite the initial warning).

**Coefficients Table:**
- Coef.: The estimated effect sizes for each fixed effect.
- Std.Err.: The standard error of each coefficient estimate.
- z: The z-statistic for testing the null hypothesis that the coefficient is zero.
- P>|z|: The p-value associated with the z-statistic.
- [0.025 0.975]: The 95% confidence interval for the coefficient estimates.

**Interpretation:**
- Intercept: The baseline level of the dependent variable when all predictors are at their reference levels.
- Time_Point and Group: The estimated effects of different time points and groups on the dependent variable.
- Group Var: The estimated variance associated with the random effects (Animal_ID).

**Important Points:**
- The model has estimated effects for each time point within each group, as well as interactions between time points and groups.
- Some coefficients do not have associated standard errors or p-values, which might be due to the singular covariance matrix or other issues with the data.
- The p-values for some of the coefficients are significant (less than 0.05), suggesting that those time points or group interactions have a statistically significant effect on the dependent variable.
  
**Next Steps:**
Given the warnings and some missing information in the output, may need to:
- Check the data for any issues that could affect the model (e.g., outliers, missing data).
- Consider simplifying the model if it's overparameterized.
- Consult with a statistician to ensure the model is specified correctly and to interpret the results in the context of the study.
- If convergence issues persist, you might need to adjust the optimization settings or consider alternative modeling approaches.

In [120]:
result = AlHf.run_kruskal_wallis_test(df_long, groups='Group', values='value')
print(result)

KruskalResult(statistic=24.827273570428428, pvalue=1.677920008189288e-05)


Checking for normality after log normalization

In [ ]:
# Function to apply Shapiro-Wilk test to each group
def test_normality(group):
    stat, p = shapiro(group['Log_Normalized_Value'])
    return pd.Series({'W': stat, 'p': p})

# Group the data by 'Time_Point', 'Group', and 'Motif' and apply the test
normality_results_vsSham_Group = df_long_logNorm_sham.groupby(['Time_Point', 'Group', 'Motif']).apply(test_normality).reset_index()
normality_results_vsBase_Group = df_long_logNorm_base.groupby(['Time_Point', 'Group', 'Motif']).apply(test_normality).reset_index()

normality_results_vsSham_State = df_long_logNorm_sham.groupby(['Time_Point', 'Treatment_State', 'Motif']).apply(test_normality).reset_index()
normality_results_vsBase_State = df_long_logNorm_base.groupby(['Time_Point', 'Treatment_State', 'Motif']).apply(test_normality).reset_index()

In [312]:
normal_motifs_vsSham_Group = normality_results_vsSham_Group[normality_results_vsSham_Group['p'] > 0.05]
non_normal_motifs_vsSham_Group = normality_results_vsSham_Group[normality_results_vsSham_Group['p'] <= 0.05]
percent_normal_vsSham_Group = len(normal_motifs_vsSham_Group) / len(normality_results_vsSham_Group) * 100

normal_motifs_vsBase_Group = normality_results_vsBase_Group[normality_results_vsBase_Group['p'] > 0.05]
non_normal_motifs_vsBase_Group = normality_results_vsBase_Group[normality_results_vsBase_Group['p'] <= 0.05]
percent_normal_vsBase_Group = len(normal_motifs_vsBase_Group) / len(normality_results_vsBase_Group) * 100

ic(percent_normal_vsSham_Group)
ic(percent_normal_vsBase_Group)

ic| percent_normal_vsSham_Group: 89.57264957264958
ic| percent_normal_vsBase_Group: 93.58974358974359


93.58974358974359

In [313]:
normal_motifs_vsSham_State = normality_results_vsSham_State[normality_results_vsSham_State['p'] > 0.05]
non_normal_motifs_vsSham_State = normality_results_vsSham_State[normality_results_vsSham_State['p'] <= 0.05]
percent_normal_vsSham_State = len(normal_motifs_vsSham_State) / len(normality_results_vsSham_State) * 100

normal_motifs_vsBase_State = normality_results_vsBase_State[normality_results_vsBase_State['p'] > 0.05]
non_normal_motifs_vsBase_State = normality_results_vsBase_State[normality_results_vsBase_State['p'] <= 0.05]
percent_normal_vsBase_State = len(normal_motifs_vsBase_State) / len(normality_results_vsBase_State) * 100

ic(percent_normal_vsSham_State)
ic(percent_normal_vsBase_State)

ic| percent_normal_vsSham_State: 88.38383838383838
ic| percent_normal_vsBase_State: 92.82828282828282


92.82828282828282

Based on the Shapiro-Wilk test , it appears that the majority of the motif usages for each group (and state) at each time point do not significantly deviate from a normal distribution. This is indicated by the p-values being greater than the typical alpha level of 0.05.

Given that the non-normal cases are relatively few compared to the normal ones, it does suggest that almost all of the motif usages are normal. However, for those motifs that are not normally distributed, will need use non-parametric statistical tests or consider data transformation techniques if parametric tests are required for your analysis.

In [314]:
# Examining the non-normal motifs more:
# Group by 'Time_Point' and 'Group' to see which motifs are not normal in each case
grouped_non_normal_motifs_Group = non_normal_motifs_vsSham_Group.groupby(['Time_Point', 'Group'])

# Iterate over each group and print the motifs that are not normally distributed
for (time_point, group), motifs in grouped_non_normal_motifs_Group:
    print(f"Time Point: {time_point}, Group: {group}")
    print(motifs[['Motif', 'W', 'p']])
    print("\n")

Time Point: Baseline_1, Group: ABX
    Motif         W         p
3       3  0.854541  0.041790
12     12  0.736465  0.001937
13     13  0.805416  0.010803


Time Point: Baseline_1, Group: Injured
    Motif         W         p
35      5  0.847740  0.034432
41     11  0.759385  0.003355
42     12  0.831113  0.021627


Time Point: Baseline_1, Group: Treated
     Motif         W         p
90       0  0.852460  0.039377
99       9  0.828578  0.020169
100     10  0.839822  0.027550
102     12  0.850229  0.036953
103     13  0.850584  0.037328
104     14  0.855709  0.043212
105     15  0.804713  0.010605
108     18  0.843479  0.030528


Time Point: Baseline_2, Group: ABX
     Motif         W         p
131     11  0.798395  0.008984
132     12  0.763296  0.003692
133     13  0.736121  0.001922
135     15  0.831795  0.022038


Time Point: Baseline_2, Group: Injured
     Motif         W         p
156      6  0.837713  0.025973
165     15  0.735604  0.001898


Time Point: Baseline_2, Group: Treat

In [315]:
# Creating a column in the sham normalized data frame to denote when a combination of time point, group, and motif aren't normally distributed:
# Create a dictionary to map the normality status
normality_vsSham_dict_Group = {(row['Time_Point'], row['Group'], row['Motif']): row['p'] > 0.05 for index, row in normality_results_vsSham_Group.iterrows()}
normality_vsBase_dict_Group = {(row['Time_Point'], row['Group'], row['Motif']): row['p'] > 0.05 for index, row in normality_results_vsBase_Group.iterrows()}

normality_vsSham_dict_State = {(row['Time_Point'], row['Treatment_State'], row['Motif']): row['p'] > 0.05 for index, row in normality_results_vsSham_State.iterrows()}
normality_vsBase_dict_State = {(row['Time_Point'], row['Treatment_State'], row['Motif']): row['p'] > 0.05 for index, row in normality_results_vsBase_State.iterrows()}

In [317]:
# Function to apply the normality status based on the dictionary for Group
def apply_normality_status_Group(row, normality_dict):
    return normality_dict.get((row['Time_Point'], row['Group'], row['Motif']), True)  # Default to True if not found

# Apply the function to create a new column 'Is_Normal_Group'
df_long_logNorm_sham['Is_Normal_Group'] = df_long_logNorm_sham.apply(apply_normality_status_Group, axis=1, normality_dict=normality_vsSham_dict_Group)
df_long_logNorm_base['Is_Normal_Group'] = df_long_logNorm_base.apply(apply_normality_status_Group, axis=1, normality_dict=normality_vsBase_dict_Group)

# Function to apply the normality status based on the dictionary for Treatment_State
def apply_normality_status_State(row, normality_dict):
    return normality_dict.get((row['Time_Point'], row['Treatment_State'], row['Motif']), True)  # Default to True if not found

# Apply the function to create a new column 'Is_Normal_State'
df_long_logNorm_sham['Is_Normal_State'] = df_long_logNorm_sham.apply(apply_normality_status_State, axis=1, normality_dict=normality_vsSham_dict_State)
df_long_logNorm_base['Is_Normal_State'] = df_long_logNorm_base.apply(apply_normality_status_State, axis=1, normality_dict=normality_vsBase_dict_State)

In [318]:
df_long_logNorm_sham

,Animal_ID,Time_Point,Group,Treatment_State,Motif,value,Cluster,Baseline_Mean,Normalized_Value,log_value,Sham_Log_Mean,Log_Normalized_Value,Is_Normal_Group,Is_Normal_State
0,A1,Baseline_1,Sham,Sham,0,424,5,436.0,1.000000,6.052089,6.824923,0.000000,True,True
1,A1,Baseline_1,Sham,Sham,1,105,1,73.5,1.000000,4.663439,5.896740,0.000000,True,True
2,A1,Baseline_1,Sham,Sham,2,764,3,1005.5,1.000000,6.639876,6.600722,0.000000,True,True
3,A1,Baseline_1,Sham,Sham,3,199,1,161.0,1.000000,5.298317,5.864924,0.000000,True,True
4,A1,Baseline_1,Sham,Sham,4,696,3,657.5,1.000000,6.546785,6.420746,0.000000,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,Y2,Drug_Trt,Treated,Treated,25,940,7,1972.5,0.476553,6.846943,7.143300,-0.296357,True,True
14036,Y2,Drug_Trt,Treated,Treated,26,796,10,1700.0,0.468235,6.680855,6.565693,0.115162,True,True
14037,Y2,Drug_Trt,Treated,Treated,27,134,3,26.0,5.153846,4.905275,3.710625,1.194649,False,False
14038,Y2,Drug_Trt,Treated,Treated,28,738,9,916.5,0.805237,6.605298,6.536103,0.069195,True,True


In [319]:
df_long_logNorm_base

,Animal_ID,Time_Point,Group,Treatment_State,Motif,value,Cluster,Baseline_Mean,Normalized_Value,log_value,Baseline_Log_Mean,Log_Normalized_Value,Is_Normal_Group,Is_Normal_State
0,A1,Baseline_1,Sham,Sham,0,424,5,436.0,1.000000,6.052089,6.079556,0.000000,True,True
1,A1,Baseline_1,Sham,Sham,1,105,1,73.5,1.000000,4.663439,4.212320,0.000000,True,True
2,A1,Baseline_1,Sham,Sham,2,764,3,1005.5,1.000000,6.639876,6.884587,0.000000,True,True
3,A1,Baseline_1,Sham,Sham,3,199,1,161.0,1.000000,5.298317,5.059299,0.000000,True,True
4,A1,Baseline_1,Sham,Sham,4,696,3,657.5,1.000000,6.546785,6.488252,0.000000,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,Y2,Drug_Trt,Treated,Treated,25,940,7,1972.5,0.476553,6.846943,7.567698,-0.720755,True,True
14036,Y2,Drug_Trt,Treated,Treated,26,796,10,1700.0,0.468235,6.680855,7.438958,-0.758103,True,True
14037,Y2,Drug_Trt,Treated,Treated,27,134,3,26.0,5.153846,4.905275,3.001944,1.903331,True,True
14038,Y2,Drug_Trt,Treated,Treated,28,738,9,916.5,0.805237,6.605298,6.821610,-0.216312,False,False


In [351]:
reload(an)
pVals_longNorm_sham_Group = an.calculate_p_values_vs_sham_Group(df_long_logNorm_sham, log_comp = True)
pVals_longNorm_base_Group = an.calculate_p_values_vs_baseline_Group(df_long_logNorm_base, log_comp = True)
pVals_longNorm_sham_State = an.calculate_p_values_vs_sham_State(df_long_logNorm_sham, log_comp = True)
pVals_longNorm_base_State = an.calculate_p_values_vs_baseline_State(df_long_logNorm_base, log_comp = True)

In [ ]:
from statsmodels.stats.multitest import multipletests

# Adjusting p-values for the multiple tests:
p_vals = pVals_longNorm_sham_Group['P_Value'].values
adjusted_p_vals = multipletests(p_vals, alpha=0.05, method='bonferroni')


adjusted_p_vals[1]
# Add the adjusted p-values to the DataFrame
#pVals_longNorm_sham_Group['Adjusted_P_Value'] = adjusted_p_vals[1]

In [ ]:
p_vals = pVals_longNorm_base_Group['P_Value'].values
adjusted_p_vals = multipletests(p_vals, alpha=0.05, method='bonferroni')

# Add the adjusted p-values to the DataFrame
#pVals_longNorm_base_Group['Adjusted_P_Value'] = adjusted_p_vals[1]

In [ ]:
p_vals = pVals_longNorm_sham_State['P_Value'].values
adjusted_p_vals = multipletests(p_vals, alpha=0.05, method='bonferroni')

# Add the adjusted p-values to the DataFrame
#pVals_longNorm_sham_State['Adjusted_P_Value'] = adjusted_p_vals[1]

In [ ]:
p_vals = pVals_longNorm_sham_State['P_Value'].values
adjusted_p_vals = multipletests(p_vals, alpha=0.05, method='bonferroni')

# Add the adjusted p-values to the DataFrame
#pVals_longNorm_sham_State['Adjusted_P_Value'] = adjusted_p_vals[1]

In [374]:
pVals_longNorm_sham_Group

,Animal_ID,Time_Point,Group,Motif,P_Value,Stat_Value,Test_Type
0,B1,Baseline_1,ABX,0,0.724564,-0.361513,t-test
1,B1,Baseline_1,ABX,1,0.658289,0.454526,t-test
2,B1,Baseline_1,ABX,2,0.679041,0.424986,t-test
3,B1,Baseline_1,ABX,3,0.028503,108.000000,Mann-Whitney
4,B1,Baseline_1,ABX,4,0.224695,1.286470,t-test
...,...,...,...,...,...,...,...
865,J1,Drug_Trt,Treated,25,0.523923,-0.658236,t-test
866,J1,Drug_Trt,Treated,26,0.598729,-0.541846,t-test
867,J1,Drug_Trt,Treated,27,0.028464,108.000000,Mann-Whitney
868,J1,Drug_Trt,Treated,28,0.093105,1.838587,t-test


In [360]:
pVals_longNorm_base_Group

,Animal_ID,Time_Point,Group,Motif,P_Value,Stat_Value,Test_Type
0,A1,Week_02,Sham,0,0.168867,-1.472674,t-test
1,A1,Week_04,Sham,0,0.029836,-2.493752,t-test
2,A1,Week_06,Sham,0,0.000235,-5.347185,t-test
3,A1,Week_08,Sham,0,0.001135,-4.360626,t-test
4,A1,Week_11,Sham,0,0.000002,24.000000,Mann-Whitney
...,...,...,...,...,...,...,...
925,J1,Week_08,Treated,29,0.887676,-0.144557,t-test
926,J1,Week_11,Treated,29,0.134296,1.616421,t-test
927,J1,Week_13,Treated,29,0.570718,0.584438,t-test
928,J1,Week_15,Treated,29,0.216773,1.310366,t-test


In [361]:
pVals_longNorm_sham_State

,Animal_ID,Time_Point,Treatment_State,Motif,P_Value,Stat_Value,Test_Type
0,B1,Baseline_1,ABX,0,0.724564,-0.361513,t-test
1,B1,Baseline_1,ABX,1,0.658289,0.454526,t-test
2,B1,Baseline_1,ABX,2,0.679041,0.424986,t-test
3,B1,Baseline_1,ABX,3,0.028503,108.000000,Mann-Whitney
4,B1,Baseline_1,ABX,4,0.224695,1.286470,t-test
...,...,...,...,...,...,...,...
685,J1,Drug_Trt,Treated,25,0.523923,-0.658236,t-test
686,J1,Drug_Trt,Treated,26,0.598729,-0.541846,t-test
687,J1,Drug_Trt,Treated,27,0.028464,108.000000,Mann-Whitney
688,J1,Drug_Trt,Treated,28,0.093105,1.838587,t-test


In [362]:
pVals_longNorm_base_State

,Animal_ID,Time_Point,Treatment_State,Motif,P_Value,Stat_Value,Test_Type
0,A1,Week_02,Sham,0,1.688665e-01,-1.472674,t-test
1,A1,Week_04,Sham,0,2.983584e-02,-2.493752,t-test
2,A1,Week_06,Sham,0,2.348192e-04,-5.347185,t-test
3,A1,Week_08,Sham,0,1.135112e-03,-4.360626,t-test
4,A1,Week_11,Sham,0,1.757720e-06,24.000000,Mann-Whitney
...,...,...,...,...,...,...,...
805,J1,Drug_Trt,Treated,28,2.846117e-14,0.000000,Mann-Whitney
806,J1,Week_11,Treated,29,1.342959e-01,1.616421,t-test
807,J1,Week_13,Treated,29,5.707177e-01,0.584438,t-test
808,J1,Week_15,Treated,29,2.167733e-01,1.310366,t-test


In [364]:
# Merge the group p-values
df_long_logNorm_sham = df_long_logNorm_sham.merge(
    pVals_longNorm_sham_Group[['Time_Point', 'Group', 'Motif', 'P_Value', 'Stat_Value', 'Test_Type']],
    on=['Time_Point', 'Group', 'Motif'],
    how='left',
    suffixes=('', '_group')
)

# Merge the state p-values
df_long_logNorm_sham = df_long_logNorm_sham.merge(
    pVals_longNorm_sham_State[['Time_Point', 'Treatment_State', 'Motif', 'P_Value', 'Stat_Value', 'Test_Type']],
    on=['Time_Point', 'Treatment_State', 'Motif'],
    how='left',
    suffixes=('', '_state')
)

# Rename the columns to make them differentiable
df_long_logNorm_sham.rename(columns={
    'P_Value': 'P_Value_Group',
    'Stat_Value': 'Stat_Value_Group',
    'Test_Type': 'Test_Type_Group',
    'P_Value_state': 'P_Value_State',
    'Stat_Value_state': 'Stat_Value_State',
    'Test_Type_state': 'Test_Type_State'
}, inplace=True)

df_long_logNorm_sham

,Animal_ID,Time_Point,Group,Treatment_State,Motif,value,Cluster,Baseline_Mean,Normalized_Value,log_value,Sham_Log_Mean,Log_Normalized_Value,Is_Normal_Group,Is_Normal_State,P_Value_Group,Stat_Value_Group,Test_Type_Group,P_Value_State,Stat_Value_State,Test_Type_State
0,A1,Baseline_1,Sham,Sham,0,424,5,436.0,1.000000,6.052089,6.824923,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,Baseline_1,Sham,Sham,1,105,1,73.5,1.000000,4.663439,5.896740,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,A1,Baseline_1,Sham,Sham,2,764,3,1005.5,1.000000,6.639876,6.600722,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,A1,Baseline_1,Sham,Sham,3,199,1,161.0,1.000000,5.298317,5.864924,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,A1,Baseline_1,Sham,Sham,4,696,3,657.5,1.000000,6.546785,6.420746,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,Y2,Drug_Trt,Treated,Treated,25,940,7,1972.5,0.476553,6.846943,7.143300,-0.296357,True,True,0.523923,-0.658236,t-test,0.523923,-0.658236,t-test
14036,Y2,Drug_Trt,Treated,Treated,26,796,10,1700.0,0.468235,6.680855,6.565693,0.115162,True,True,0.598729,-0.541846,t-test,0.598729,-0.541846,t-test
14037,Y2,Drug_Trt,Treated,Treated,27,134,3,26.0,5.153846,4.905275,3.710625,1.194649,False,False,0.028464,108.000000,Mann-Whitney,0.028464,108.000000,Mann-Whitney
14038,Y2,Drug_Trt,Treated,Treated,28,738,9,916.5,0.805237,6.605298,6.536103,0.069195,True,True,0.093105,1.838587,t-test,0.093105,1.838587,t-test


In [365]:
# Merge the group p-values
df_long_logNorm_base = df_long_logNorm_base.merge(
    pVals_longNorm_base_Group[['Time_Point', 'Group', 'Motif', 'P_Value', 'Stat_Value', 'Test_Type']],
    on=['Time_Point', 'Group', 'Motif'],
    how='left',
    suffixes=('', '_group')
)

# Merge the state p-values
df_long_logNorm_base = df_long_logNorm_base.merge(
    pVals_longNorm_base_State[['Time_Point', 'Treatment_State', 'Motif', 'P_Value', 'Stat_Value', 'Test_Type']],
    on=['Time_Point', 'Treatment_State', 'Motif'],
    how='left',
    suffixes=('', '_state')
)

# Rename the columns to make them differentiable
df_long_logNorm_base.rename(columns={
    'P_Value': 'P_Value_Group',
    'Stat_Value': 'Stat_Value_Group',
    'Test_Type': 'Test_Type_Group',
    'P_Value_state': 'P_Value_State',
    'Stat_Value_state': 'Stat_Value_State',
    'Test_Type_state': 'Test_Type_State'
}, inplace=True)

df_long_logNorm_base

,Animal_ID,Time_Point,Group,Treatment_State,Motif,value,Cluster,Baseline_Mean,Normalized_Value,log_value,Baseline_Log_Mean,Log_Normalized_Value,Is_Normal_Group,Is_Normal_State,P_Value_Group,Stat_Value_Group,Test_Type_Group,P_Value_State,Stat_Value_State,Test_Type_State
0,A1,Baseline_1,Sham,Sham,0,424,5,436.0,1.000000,6.052089,6.079556,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,Baseline_1,Sham,Sham,1,105,1,73.5,1.000000,4.663439,4.212320,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,A1,Baseline_1,Sham,Sham,2,764,3,1005.5,1.000000,6.639876,6.884587,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,A1,Baseline_1,Sham,Sham,3,199,1,161.0,1.000000,5.298317,5.059299,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,A1,Baseline_1,Sham,Sham,4,696,3,657.5,1.000000,6.546785,6.488252,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,Y2,Drug_Trt,Treated,Treated,25,940,7,1972.5,0.476553,6.846943,7.567698,-0.720755,True,True,7.478622e-01,0.329642,t-test,7.478622e-01,0.329642,t-test
14036,Y2,Drug_Trt,Treated,Treated,26,796,10,1700.0,0.468235,6.680855,7.438958,-0.758103,True,True,7.269034e-04,-4.631269,t-test,7.269034e-04,-4.631269,t-test
14037,Y2,Drug_Trt,Treated,Treated,27,134,3,26.0,5.153846,4.905275,3.001944,1.903331,True,True,2.200776e-01,-1.300313,t-test,2.200776e-01,-1.300313,t-test
14038,Y2,Drug_Trt,Treated,Treated,28,738,9,916.5,0.805237,6.605298,6.821610,-0.216312,False,False,9.504392e-09,0.000000,Mann-Whitney,2.846117e-14,0.000000,Mann-Whitney


In [368]:
# Count the number of significant motifs for each Group at each Time_Point when comparing to sham
# and list the significant motifs
significant_counts_vsSham_Group = pVals_longNorm_sham_Group[pVals_longNorm_sham_Group['P_Value'] <= 0.05].groupby(['Time_Point', 'Group'])['Motif'].apply(list).reset_index(name='Significant_Motifs_Group')
significant_counts_vsSham_Group['Significant_Motif_Count_Group'] = significant_counts_vsSham_Group['Significant_Motifs_Group'].apply(len)

significant_counts_vsBase_Group = pVals_longNorm_base_Group[pVals_longNorm_base_Group['P_Value'] <= 0.05].groupby(['Time_Point', 'Group'])['Motif'].apply(list).reset_index(name='Significant_Motifs_Group')
significant_counts_vsBase_Group['Significant_Motif_Count_Group'] = significant_counts_vsBase_Group['Significant_Motifs_Group'].apply(len)

significant_counts_vsSham_State = pVals_longNorm_sham_State[pVals_longNorm_sham_State['P_Value'] <= 0.05].groupby(['Time_Point', 'Treatment_State'])['Motif'].apply(list).reset_index(name='Significant_Motifs_State')
significant_counts_vsSham_State['Significant_Motif_Count_State'] = significant_counts_vsSham_State['Significant_Motifs_State'].apply(len)

significant_counts_vsBase_State = pVals_longNorm_base_State[pVals_longNorm_base_State['P_Value'] <= 0.05].groupby(['Time_Point', 'Treatment_State'])['Motif'].apply(list).reset_index(name='Significant_Motifs_State')
significant_counts_vsBase_State['Significant_Motif_Count_State'] = significant_counts_vsBase_State['Significant_Motifs_State'].apply(len)


In [500]:
significant_counts_vsSham_Group
significant_counts_vsSham_Group.to_csv('kmeans-30_significant_counts_vsSham_Group.csv', index=False)

In [501]:
significant_counts_vsSham_State
significant_counts_vsSham_State.to_csv('kmeans-30_significant_counts_vsSham_State.csv', index=False)

In [503]:
significant_counts_vsBase_Group
significant_counts_vsBase_Group.to_csv('kmeans-30_significant_counts_vsBase_Group.csv', index=False)

In [504]:
significant_counts_vsBase_State
significant_counts_vsBase_State.to_csv('kmeans-30_significant_counts_vsBase_State.csv', index=False)

In [485]:
reload(AlPf)
AlPf.plot_motifs_all_time_points(df_long_logNorm_sham, comparison = 'Group')

In [506]:
reload(AlPf)
AlPf.plot_motifs_all_time_points(df_long_logNorm_sham, comparison = 'Treatment_State')

KeyboardInterrupt: 

In [401]:
reload(AlPf)
AlPf.plot_motifs4(df_long_logNorm_sham, comparison = 'Group')

In [392]:
reload(AlPf)
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Baseline_1", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Baseline_2", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_02", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_04", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_06", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_08", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_11", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_13", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_15", comparison = 'Group')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Drug_Trt", comparison = 'Group')






In [ ]:
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Baseline_1", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Baseline_2", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_02", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_04", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_06", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_08", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_11", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_13", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_15", comparison = 'Treatment_State')
AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Drug_Trt", comparison = 'Treatment_State')



AlPf.plot_motifs3(df_long_logNorm_sham, time_point="Week_08", comparison = 'Group')

AlPf.plot_motifs3(df_long_logNorm_base, time_point="Week_08", comparison = 'Group')


In [385]:
AlPf.plot_motifs2(df_long_logNorm_base, time_point="Week_15", comparison = 'Group')
AlPf.plot_motifs2(df_long_logNorm_base, time_point="Week_15", comparison = 'Treatment_State')

In [375]:
df_long_logNorm_sham

,Animal_ID,Time_Point,Group,Treatment_State,Motif,value,Cluster,Baseline_Mean,Normalized_Value,log_value,Sham_Log_Mean,Log_Normalized_Value,Is_Normal_Group,Is_Normal_State,P_Value_Group,Stat_Value_Group,Test_Type_Group,P_Value_State,Stat_Value_State,Test_Type_State
0,A1,Baseline_1,Sham,Sham,0,424,5,436.0,1.000000,6.052089,6.824923,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,Baseline_1,Sham,Sham,1,105,1,73.5,1.000000,4.663439,5.896740,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,A1,Baseline_1,Sham,Sham,2,764,3,1005.5,1.000000,6.639876,6.600722,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,A1,Baseline_1,Sham,Sham,3,199,1,161.0,1.000000,5.298317,5.864924,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,A1,Baseline_1,Sham,Sham,4,696,3,657.5,1.000000,6.546785,6.420746,0.000000,True,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,Y2,Drug_Trt,Treated,Treated,25,940,7,1972.5,0.476553,6.846943,7.143300,-0.296357,True,True,0.523923,-0.658236,t-test,0.523923,-0.658236,t-test
14036,Y2,Drug_Trt,Treated,Treated,26,796,10,1700.0,0.468235,6.680855,6.565693,0.115162,True,True,0.598729,-0.541846,t-test,0.598729,-0.541846,t-test
14037,Y2,Drug_Trt,Treated,Treated,27,134,3,26.0,5.153846,4.905275,3.710625,1.194649,False,False,0.028464,108.000000,Mann-Whitney,0.028464,108.000000,Mann-Whitney
14038,Y2,Drug_Trt,Treated,Treated,28,738,9,916.5,0.805237,6.605298,6.536103,0.069195,True,True,0.093105,1.838587,t-test,0.093105,1.838587,t-test


In [190]:
reload(AlPf)
reload(an)

AlPf.plot_boxplot_by_group_and_timepoint_2(df_long_logNorm_sham, significance_level=0.05, specific_time_point="Baseline_1", log_norm=True)

07:16:34 PM INFO:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
07:16:34 PM INFO:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


       Time_Point    Group  Motif  Significant_Motif_Count  Cluster
0        Drug_Trt  Injured      0                        1        5
468      Drug_Trt  Injured      2                        1        3
936      Drug_Trt  Injured      3                        1        1
1404     Drug_Trt  Injured      4                        1        3
1872     Drug_Trt  Injured      5                        1        7
...           ...      ...    ...                      ...      ...
157248    Week_15  Treated     21                        1        5
157716    Week_15  Treated     23                        1        3
158184    Week_15  Treated     24                        1        3
158652    Week_15  Treated     26                        1       10
159120    Week_15  Treated     28                        1        9

[341 rows x 5 columns]


In [189]:
reload(AlPf)
reload(an)
pVals_longNorm_base = an.calculate_p_values_vs_sham(df_long_logNorm_base, test = 'u_stat', log_comp = True)
AlPf.plot_normalized_values_by_group_and_timepoint_3(pVals_longNorm_base, df_long_logNorm_base, significance_level=0.05, specific_time_point=None, log_norm=True)

In [502]:
pVals_longNorm_base

,Animal_ID,Time_Point,Group,Motif,P_Value,Stat_Value,Test_Type
0,A1,Week_02,Sham,0,0.168867,-1.472674,t-test
1,A1,Week_04,Sham,0,0.029836,-2.493752,t-test
2,A1,Week_06,Sham,0,0.000235,-5.347185,t-test
3,A1,Week_08,Sham,0,0.001135,-4.360626,t-test
4,A1,Week_11,Sham,0,0.000002,24.000000,Mann-Whitney
...,...,...,...,...,...,...,...
925,J1,Week_08,Treated,29,0.887676,-0.144557,t-test
926,J1,Week_11,Treated,29,0.134296,1.616421,t-test
927,J1,Week_13,Treated,29,0.570718,0.584438,t-test
928,J1,Week_15,Treated,29,0.216773,1.310366,t-test


In [177]:
master_df = pd.read_csv("/Users/adanredwine/Desktop/ALR_1-VAME-Project-Feb3-2023/all_sequences_kmeans-30.csv")

In [188]:
reload(AlPf)
time_point = "Week_15"
AlPf.plot_motif_frequency(master_df, time_point, motifs=None)

07:16:14 PM INFO:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
07:16:14 PM INFO:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


       group  motif  frequency
0        ABX      0      10080
1        ABX      1       6120
2        ABX      2       7326
3        ABX      3       5676
4        ABX      4      10926
..       ...    ...        ...
115  Treated     25      15642
116  Treated     26       9386
117  Treated     27       2673
118  Treated     28       9447
119  Treated     29      18106

[120 rows x 3 columns]


# Kinematics Analysis

## Total Distance Traveled
### The Plan:
Step 1: Data Preparation
    - 1.1. Load the dataset into a Polars DataFrame if not already loaded.
    - 1.2. Verify the integrity of the 'distance' column (no missing or aberrant values).
    - 1.3. Group the data by 'group' and 'time_point' to prepare for aggregation.
  
Step 2: Baseline Calculation
    - 2.1. Filter the dataset for 'Baseline_1' and 'Baseline_2' time points.
    - 2.2. Calculate the average distance and standard deviation for these two baseline time points for each 'group'.
    - 2.3. Store these baseline averages in a separate DataFrame or dictionary for later use.
  
Step 3: Normality Checks
    - 3.1. For each 'group' and 'time_point', perform normality tests (e.g., Shapiro-Wilk) on the 'distance' data.
    - 3.2. Use Q-Q plots and histograms for visual inspection of normality.
    - 3.3. Document the results to determine the appropriate statistical tests.

Step 4: Total Distance Calculation
    - 4.1. Aggregate the total distance traveled by summing the 'distance' column for each 'group' and 'time_point'.
    - 4.2. Store these totals in a new DataFrame with 'group', 'time_point', and 'total_distance' columns.

Step 5: Statistical Comparisons (Pre-Normalization)
    - 5.1. For each 'group' and 'time_point', compare the total distance to the group's baseline average using the appropriate statistical test based on normality results.
    - 5.2. Adjust for multiple comparisons if necessary.
    - 5.3. Document the results, including p-values and effect sizes.

Step 6: Normalization
    - 6.1. Normalize the total distance for each 'group' and 'time_point' to their respective baseline averages.
    - 6.2. Normalize the total distance for each 'group' and 'time_point' to the 'Sham' group's values at the same time points.
    - 6.3. Handle any zero or near-zero baseline values carefully to avoid division by zero.

Step 7: Normality Checks (Post-Normalization)
    - 7.1. Re-check the normalized data for normality using the same methods as in Step 3.
    - 7.2. Document any changes in the distribution due to normalization.

Step 8: Statistical Comparisons (Post-Normalization)
    - 8.1. Repeat the statistical comparisons from Step 5 using the normalized data.
    - 8.2. Adjust for multiple comparisons if necessary.
    - 8.3. Document the results, including p-values and effect sizes.

Step 9: Interpretation and Reporting
    - 9.1. Interpret the results from both pre- and post-normalization analyses.
    - 9.2. Discuss any significant findings and their implications.
    - 9.3. Prepare a report or presentation summarizing the methodology, analyses, and conclusions.

Step 10: Review and Validation
    - 10.1. Review all steps and results for accuracy and completeness.
    - 10.2. Validate the findings by cross-checking with other data or analyses if available.
    - 10.3. Ensure that all assumptions for statistical tests are met and documented.
